# Data Science project

This project explores the MineCraft server and player data collected by the Frank Woods reseach group in Computer Science at UBC to explore if certain player charachteristics are predictive of a newsletter subscription. 

https://plai.cs.ubc.ca/
https://www.cs.ubc.ca/~fwood/



# Question: Is there a relationship between hours played and age that predicts wheather a player has a newsletter subscription?

Variables in the `player.csv` data set: 

- experience: a skill ranking ascending from begeinner, amature, regular, pro and veteran
- subscribe: whether an individual is subscribed or not to the newsletter
- hashedEmail: encrypted email address
- played_hours: total played hours range mmead and sd
- name: first name
- gender: individuals gender list
- Age: individuals gender 



In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

Data Exploration and Visualization

The first steps is to read the data and mutate the response variable into a factor to tell R that this is a classification problem.

In [ ]:
player_data <- read_csv("data/players.csv") |>
    mutate(subscribe = as_factor(subscribe))
player_data

In [ ]:
select_player_data <- player_data |> 
    select(subscribe, Age, played_hours)
select_player_data

summary(select_player_data)

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 9)

player_data_plot <- select_player_data |>
    ggplot(aes(x = played_hours, y = Age, color = subscribe)) +
    geom_point(alpha = 0.4) +
    labs(x = "Number of hours played", 
         y = "Age", 
         title = "Figure 1 Exploring relationship b/w Age and Number of hours played on subscription",
        color = "subsciption") +  
    theme(text = element_text(size = 13))
player_data_plot 

From Figure 1. we can clearly see that all individuals with a high number of 25+ hours played have newsletter subscription. However, individuals with 0 hours vary in subscriptions 

# Data Analysis
To determine if there's a relationship between hours played and age that predicts whether a player has a subscription, I approached this question as a classification problem.

`subscribe` has 2 categories `TRUE`(subscribed) or `FALSE`(unsubscribed) and as the outcome variable we turned it into a factor in the previous lines of code.

Split data and evaluate proportions 

In [ ]:
set.seed(111)

player_split <- initial_split(player_data, prop = 0.75, strata = subscribe)
player_train <- training(player_split)
player_test <- testing(player_split)

player_train <- player_train |>
    drop_na()
player_test <- player_test |>
    drop_na()

table(player_train$subscribe)
table(player_test$subscribe)

player_train_proportions <- player_train |>
                      group_by(subscribe) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(player_train))

player_train_proportions

Looking at the proportions in the training data, we need to keep in mind that there isnt an equal amount of people subscribed and unsubscribed so we want to make sure we calculate recall (to catch as many sctual subscibers) and precision (to be correct most of the time)

predicting labels in the test set and evaluating our classifier's performance

Cross - validation

In [ ]:
set.seed(111)

player_vfold <- vfold_cv(player_train, v = 5, strata = subscribe)


player_recipe <- recipe(subscribe ~ Age + played_hours,
                        data = player_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

k_vals <- tibble(neighbors = seq(from = 1, to = 30, by = 2))

knn_results <- workflow() |>
  add_recipe(player_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = player_vfold, grid = k_vals) |>
  collect_metrics()

accuracies <- knn_results |>
  filter(.metric == "accuracy")

accuracies


In [ ]:
set.seed(111)

accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", 
       y = "Accuracy Estimate",
      title = "Figure 2. K neighbors VS accuracy estimate") +
  theme(text = element_text(size = 12))

accuracy_vs_k

best_k <- accuracies |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
best_k

in this case the best number of neighbors from a  was k = 19 with a round 74% accuracy.

Now we will evaluate on the test set

In [ ]:
set.seed(111)

player_recipe <- recipe(subscribe ~ Age + played_hours, data = player_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

knn_spec2 <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit2 <- workflow() |>
  add_recipe(player_recipe) |>
  add_model(knn_spec2) |>
  fit(data = player_train)

knn_fit2

Now we will look at accuracy, recall and precision to see if there any improvement

In [ ]:
set.seed(111)

player_test_predictions2 <- predict(knn_fit2, player_test) |>
  bind_cols(player_test)

player_test_predictions2 |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")


player_test_predictions2 |>
    precision(truth = subscribe, estimate = .pred_class, event_level="first")

player_test_predictions2 |>
    recall(truth = subscribe, estimate = .pred_class, event_level="first")

confusion <- player_test_predictions2 |>
             conf_mat(truth = subscribe, estimate = .pred_class)
confusion

plot acuracy vs k values with the tuning results

In [ ]:
conf_tibble <- as_tibble(confusion$table)
conf_tibble



In [ ]:
bar_plot <- conf_tibble |>
    ggplot(aes(x = Truth, y = n, fill = Prediction)) +
  geom_bar(stat = "identity", position = "fill") +
  labs(
    title = "Figure 3. Confusion Matrix as Bar Plot",
    x = "ACTUAL Subscription Status",
    y = "Number of Players",
    fill = "Predicted Status by classifier"
  ) +
  theme_minimal()

bar_plot